<a href="https://colab.research.google.com/github/dekoska/machine-learning-bootcamp/blob/main/supervised/02_regression/04_backward_elimination.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [161]:
!pip install scikit-learn

In [162]:
import numpy as np
import pandas as pd
import sklearn as sk
import seaborn as sns
import matplotlib.pyplot as plt

sns.set()
np.random.seed(42)
np.set_printoptions(precision=4, suppress=True)

In [163]:
df_raw = pd.read_csv('https://storage.googleapis.com/esmartdata-courses-files/ml-course/insurance.csv')
df_raw.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [164]:
df = df_raw
df[df.duplicated()]

,age,sex,bmi,children,smoker,region,charges
581,19,male,30.59,0,no,northwest,1639.5631


In [165]:
df[df['charges'] == 1639.5631]

,age,sex,bmi,children,smoker,region,charges
195,19,male,30.59,0,no,northwest,1639.5631
581,19,male,30.59,0,no,northwest,1639.5631


In [166]:
df.drop_duplicates()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520
...,...,...,...,...,...,...,...
1333,50,male,30.970,3,no,northwest,10600.54830
1334,18,female,31.920,0,no,northeast,2205.98080
1335,18,female,36.850,0,no,southeast,1629.83350
1336,21,female,25.800,0,no,southwest,2007.94500


In [167]:
df.isnull().sum()

,0
age,0
sex,0
bmi,0
children,0
smoker,0
region,0
charges,0


In [168]:
df_dummies = pd.get_dummies(df, drop_first=True, dtype=int)
df_dummies.head()

,age,bmi,children,charges,sex_male,smoker_yes,region_northwest,region_southeast,region_southwest
0,19,27.900,0,16884.92400,0,1,0,0,1
1,18,33.770,1,1725.55230,1,0,0,1,0
2,28,33.000,3,4449.46200,1,0,0,1,0
3,33,22.705,0,21984.47061,1,0,1,0,0
4,32,28.880,0,3866.85520,1,0,1,0,0


In [169]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()  #zmienia typ na numpy array
scaler.fit(df_dummies)
df_scaled = scaler.transform(df_dummies)
df_scaled = pd.DataFrame(df_scaled, columns=df_dummies.columns)
target = df_scaled.pop('charges')

In [170]:
print(type(df_scaled))
df_scaled.head()

<class 'pandas.core.frame.DataFrame'>


,age,bmi,children,sex_male,smoker_yes,region_northwest,region_southeast,region_southwest
0,0.021739,0.321227,0.0,0.0,1.0,0.0,0.0,1.0
1,0.000000,0.479150,0.2,1.0,0.0,0.0,1.0,0.0
2,0.217391,0.458434,0.6,1.0,0.0,0.0,1.0,0.0
3,0.326087,0.181464,0.0,1.0,0.0,1.0,0.0,0.0
4,0.304348,0.347592,0.0,1.0,0.0,1.0,0.0,0.0


In [171]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(df_scaled,target, test_size=0.25)

In [172]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(x_train, y_train)
model.score(x_train, y_train)

0.7449555328228536

In [173]:
y_pred = model.predict(x_test)
y_pred[:10]

array([ 0.125 ,  0.0947,  0.5709,  0.1341,  0.4125,  0.1557, -0.0165,
        0.2542, -0.0018,  0.1625])

In [174]:
y_true = y_test
predictions = pd.DataFrame({'y_true': y_true, 'y_pred': y_pred})
predictions['error'] = predictions['y_true'] - predictions['y_pred']
predictions.head()

,y_true,y_pred,error
764,0.127269,0.124988,0.002280
887,0.066247,0.094686,-0.028438
890,0.450276,0.570907,-0.120631
1293,0.130570,0.134091,-0.003521
259,0.520817,0.412459,0.108358


In [175]:
from sklearn.metrics import mean_absolute_error

mae = mean_absolute_error(y_true, y_pred)
mae

0.06773746300995268

### ELIMINACJA WSTECZNA

In [176]:
import statsmodels.api as sm

x_train_ols = x_train.copy()
x_train_ols = x_train_ols.values
x_train_ols = sm.add_constant(x_train)
x_train_ols

,const,age,bmi,children,sex_male,smoker_yes,region_northwest,region_southeast,region_southwest
693,1.0,0.130435,0.207022,0.0,1.0,0.0,1.0,0.0,0.0
1297,1.0,0.217391,0.283831,0.4,0.0,0.0,0.0,1.0,0.0
634,1.0,0.717391,0.638687,0.2,1.0,0.0,0.0,0.0,1.0
1022,1.0,0.630435,0.541297,0.2,1.0,1.0,0.0,1.0,0.0
178,1.0,0.608696,0.348130,0.4,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...
1095,1.0,0.000000,0.414044,0.8,0.0,0.0,0.0,0.0,0.0
1130,1.0,0.456522,0.212806,1.0,0.0,0.0,0.0,1.0,0.0
1294,1.0,0.869565,0.247915,0.0,1.0,0.0,0.0,0.0,0.0
860,1.0,0.413043,0.851224,0.4,0.0,1.0,0.0,0.0,1.0


In [177]:
ols = sm.OLS(endog=y_train, exog=x_train_ols).fit() #ols to regresja liniowa
predictors = ['const'] + list(x_train.columns)
print(ols.summary(xname=predictors))

                            OLS Regression Results                            
Dep. Variable:                charges   R-squared:                       0.745
Model:                            OLS   Adj. R-squared:                  0.743
Method:                 Least Squares   F-statistic:                     362.9
Date:                Mon, 03 Mar 2025   Prob (F-statistic):          1.05e-288
Time:                        13:34:38   Log-Likelihood:                 915.82
No. Observations:                1003   AIC:                            -1814.
Df Residuals:                     994   BIC:                            -1769.
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const               -0.0505      0.011  

In [187]:
x_selected = x_train_ols.iloc[:, [0,1,2,3,5,6,7,8]]
predictors.remove('sex_male')

ols = sm.OLS(endog=y_train, exog=x_selected).fit()
print(ols.summary(xname=predictors))

ValueError: list.remove(x): x not in list

In [179]:
x_selected = x_train_ols.iloc[:, [0,1,2,3,4,6,7]]
predictors.remove('region_northwest')

ols = sm.OLS(endog=y_train, exog=x_selected).fit()
print(ols.summary(xname=predictors))

                            OLS Regression Results                            
Dep. Variable:                charges   R-squared:                       0.120
Model:                            OLS   Adj. R-squared:                  0.115
Method:                 Least Squares   F-statistic:                     22.67
Date:                Mon, 03 Mar 2025   Prob (F-statistic):           3.85e-25
Time:                        13:37:29   Log-Likelihood:                 294.80
No. Observations:                1003   AIC:                            -575.6
Df Residuals:                     996   BIC:                            -541.2
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const                0.0189      0.019  

In [180]:
x_selected = x_train_ols.iloc[:, [0,1,2,3,4,6]]
predictors.remove('region_southwest')

ols = sm.OLS(endog=y_train, exog=x_selected).fit()
print(ols.summary(xname=predictors))

                            OLS Regression Results                            
Dep. Variable:                charges   R-squared:                       0.120
Model:                            OLS   Adj. R-squared:                  0.115
Method:                 Least Squares   F-statistic:                     27.14
Date:                Mon, 03 Mar 2025   Prob (F-statistic):           8.40e-26
Time:                        13:38:50   Log-Likelihood:                 294.61
No. Observations:                1003   AIC:                            -577.2
Df Residuals:                     997   BIC:                            -547.8
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const                0.0203      0.019  

In [181]:
x_selected = x_train_ols.iloc[:, [0,1,2,3,4]]
predictors.remove('region_southeast')

ols = sm.OLS(endog=y_train, exog=x_selected).fit()
print(ols.summary(xname=predictors))

                            OLS Regression Results                            
Dep. Variable:                charges   R-squared:                       0.119
Model:                            OLS   Adj. R-squared:                  0.116
Method:                 Least Squares   F-statistic:                     33.82
Date:                Mon, 03 Mar 2025   Prob (F-statistic):           1.70e-26
Time:                        13:39:48   Log-Likelihood:                 294.37
No. Observations:                1003   AIC:                            -578.7
Df Residuals:                     998   BIC:                            -554.2
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0173      0.018      0.951      0.3

In [182]:
x_selected = x_train_ols.iloc[:, [1,2,3,4]]
predictors.remove('const')

ols = sm.OLS(endog=y_train, exog=x_selected).fit()
print(ols.summary(xname=predictors))

                                 OLS Regression Results                                
Dep. Variable:                charges   R-squared (uncentered):                   0.563
Model:                            OLS   Adj. R-squared (uncentered):              0.561
Method:                 Least Squares   F-statistic:                              321.7
Date:                Mon, 03 Mar 2025   Prob (F-statistic):                   7.76e-178
Time:                        13:40:22   Log-Likelihood:                          293.92
No. Observations:                1003   AIC:                                     -579.8
Df Residuals:                     999   BIC:                                     -560.2
Df Model:                           4                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

In [183]:
x_selected = x_train_ols.iloc[:, [0,1,2]]
predictors.remove('smoker_yes')

ols = sm.OLS(endog=y_train, exog=x_selected).fit()
print(ols.summary(xname=predictors))

                            OLS Regression Results                            
Dep. Variable:                charges   R-squared:                       0.113
Model:                            OLS   Adj. R-squared:                  0.111
Method:                 Least Squares   F-statistic:                     63.74
Date:                Mon, 03 Mar 2025   Prob (F-statistic):           8.86e-27
Time:                        13:41:00   Log-Likelihood:                 290.78
No. Observations:                1003   AIC:                            -575.6
Df Residuals:                    1000   BIC:                            -560.8
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
age            0.0369      0.017      2.217      0.0

### AUTOMATYZACJA TEGO WYZEJ

In [209]:
x_train_ols = x_train.copy()
x_train_ols = x_train_ols.values
x_train_ols = sm.add_constant(x_train)
x_train_ols

,const,age,bmi,children,sex_male,smoker_yes,region_northwest,region_southeast,region_southwest
693,1.0,0.130435,0.207022,0.0,1.0,0.0,1.0,0.0,0.0
1297,1.0,0.217391,0.283831,0.4,0.0,0.0,0.0,1.0,0.0
634,1.0,0.717391,0.638687,0.2,1.0,0.0,0.0,0.0,1.0
1022,1.0,0.630435,0.541297,0.2,1.0,1.0,0.0,1.0,0.0
178,1.0,0.608696,0.348130,0.4,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...
1095,1.0,0.000000,0.414044,0.8,0.0,0.0,0.0,0.0,0.0
1130,1.0,0.456522,0.212806,1.0,0.0,0.0,0.0,1.0,0.0
1294,1.0,0.869565,0.247915,0.0,1.0,0.0,0.0,0.0,0.0
860,1.0,0.413043,0.851224,0.4,0.0,1.0,0.0,0.0,1.0


In [210]:
ols = sm.OLS(endog=y_train, exog=x_train_ols).fit() #ols to regresja liniowa
predictors = ['const'] + list(x_train.columns)
print(ols.summary(xname=predictors))

                            OLS Regression Results                            
Dep. Variable:                charges   R-squared:                       0.745
Model:                            OLS   Adj. R-squared:                  0.743
Method:                 Least Squares   F-statistic:                     362.9
Date:                Mon, 03 Mar 2025   Prob (F-statistic):          1.05e-288
Time:                        14:14:45   Log-Likelihood:                 915.82
No. Observations:                1003   AIC:                            -1814.
Df Residuals:                     994   BIC:                            -1769.
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const               -0.0505      0.011  

In [212]:
while True:
    ols = sm.OLS(endog=y_train, exog=x_train_ols).fit()
    p_values = ols.pvalues.round(3)

    worst_predictor = p_values[p_values > 0.05].index

    if worst_predictor.empty:
        break

    worst_predictor = worst_predictor[0]

    if worst_predictor in predictors:
        predictors.remove(worst_predictor)

    x_train_ols = x_train_ols.drop(worst_predictor, axis=1)

print(ols.summary(xname=predictors))

                            OLS Regression Results                            
Dep. Variable:                charges   R-squared:                       0.744
Model:                            OLS   Adj. R-squared:                  0.743
Method:                 Least Squares   F-statistic:                     725.7
Date:                Mon, 03 Mar 2025   Prob (F-statistic):          1.41e-293
Time:                        14:16:08   Log-Likelihood:                 914.26
No. Observations:                1003   AIC:                            -1819.
Df Residuals:                     998   BIC:                            -1794.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0556      0.009     -5.871      0.0

In [213]:
ols.save('model.pickle')